Chapter 18 Balanced Binary Search Trees<br>

We will say that a BST with n nodes is balanced if the height of the
tree is at most some constant times log n.<br>

The most basic such operation is called a tree rotation. It comes in
two forms, rotateright and rotateleft. Rotating a node to the right will
move it to be the right child of its left child and will update the children of
these nodes appropriately.

In [1]:
def rotateright(self):
    newroot = self.left
    self.left = newroot.right
    newroot.right = self
    return newroot

Notice that rotateright returns the new root (of the subtree). This
is a very useful convention when working with BSTs and rotations. Every
method that can change the structure of the tree will return the new root of
the resulting subtree.

18.1 A BSTMapping implementation

In [2]:
from ds2.orderedmapping import BSTMapping, BSTNode

class BalancedBSTNode(BSTNode):
    def newnode(self, key, value):
        return BalancedBSTNode(key, value)
    
    def put(self, key, value):
        if key == self.key:
            self.value = value
        elif key < self.key:
            if self.left:
                self.left = self.left.put(key, value)
            else:
                self.left = self.newnode(key, value)
        elif key > self.key:
            if self.right:
                self.right = self.right.put(key, value)
            else:
                self.right = self.newnode(key, value)
        self._updatelength()
        return self
    
    def rotateright(self):
        newroot = self.left
        self.left = newroot.right
        newroot.right = self
        self._updatelength()
        newroot._updatelength()
        return newroot
    
    def rotateleft(self):
        newroot = self.right
        self.right = newroot.left
        newroot.left = self
        self._updatelength()
        newroot._updatelength()
        return newroot
    
class BalancedBST(BSTMapping):
    Node = BalancedBSTNode

    def put(self, key, value):
        if self._root:
            self._root = self._root.put(key, value)
        else:
            self._root = self.Node(key, value)

18.2 Weight Balanced Trees<br>

A node ***x*** is said to be balanced if
$$
len(x) + 1 < 4 * (min(len(x.left), len(x.right)) + 1)
$$

If some change causes a node to no longer be weight balanced, we will recover the weight balance by rotations.<br>
The rebalance method will check for the balance condition and do the appropriate rotations.

In [3]:
from ds2.orderedmapping import BalancedBST, BalancedBSTNode

class WBTreeNode(BalancedBSTNode):
    def newnode(self, key, value):
        return WBTreeNode(key, value)
    
    def toolight(self, other):
        otherlength = len(other) if other else 0
        return len(self) + 1 >= 4 * (otherlength + 1)
    
    def rebalance(self):
        if self.toolight(self.left):
            if self.toolight(self.right.right):
                self.right = self.right.rotateright()
            newroot = self.rotateleft()
        elif self.toolight(self.right):
            if self.toolight(self.left.left):
                self.left = self.left.rotateleft()
            newroot = self.rotateright()
        else:
            return self
        return newroot
    
    def put(self, key, value):
        newroot = BalancedBSTNode.put(self, key, value)
        return newroot.rebalance()

    def remove(self, key):
        newroot = BalancedBSTNode.remove(self, key)
        return newroot.rebalance() if newroot else None
    
class WBTree(BalancedBST):
    Node = WBTreeNode

The  **toolight** method is for checking if a subtree has enough nodes to
be a child of a weight balanced node. We use it both to check if the current
node is weight balanced and also to check if one rotation or two will be
required.

18.3 Height-Balanced Trees (AVL Trees)<br>

The motivation for balancing our BSTs was to keep the height small. Rather
than balancing by weight, we could also try to keep the heights of the left
and right subtrees close. In fact, we can require that these heights differ
by at most one.<br>

Often, AVL trees only keep the balance at each node rather than
the exact height, but computing heights is relatively painless.

In [4]:
from ds2.orderedmapping import BalancedBST, BalancedBSTNode

def height(node):
    return node.height if node else -1

def update(node):
    if node:
        node._updatelength()
        node._updatelength()

class AVLTreeNode(BalancedBSTNode):
    def __init__(self, key, value):
        BalancedBSTNode.__init__(self, key, value)
        self._updatelength()

    def newnode(self, key, value):
        return AVLTreeNode(key, value)
    
    def _updatelength(self):
        self.height = 1 + max(height(self.left), height(self.right))

    def balance(self):
        return height(self.right) - height(self.left)
    
    def rebalance(self):
        bal = self.balance()
        if bal == -2:
            if self.left.balance() > 0:
                self.left = self.left.rotateleft()
            newroot = self.rotateright()
        elif bal == 2:
            if self.right.balance() < 0:
                self.right = self.right.rotateright()
            newroot = self.rotateleft()
        else:
            return self
        
        update(newroot.left)
        update(newroot.right)
        update(newroot)
        return newroot
    
    def put(self, key, value):
        newroot = BalancedBSTNode.put(self, key, value)
        update(newroot)
        return newroot.rebalance()
    
    def remove(self, key):
        newroot = BalancedBSTNode.remove(self, key)
        update(newroot)
        return newroot.rebalance() if newroot else None
    
class AVLTree(BalancedBST):
    Node = AVLTreeNode

18.4 Splay Trees<br>

In a splay tree, every time we get or put an entry, its node will get
rotated all the way to the root. However, instead of rotating it directly, we
consider two steps at a time. The splayup method looks two levels down the
tree for the desired key. If its not exactly two levels down, it does nothing.<br>

A major difference from our previous implementations is that now, we
will modify the tree on calls to get. As a result, we will have to rewrite get
rather than inheriting it. Previously, get would return the desired value.
However, we want to return the new root on every operation that might
change the tree. So, which should we return? Clearly, we need that value to
return, and we also need to not break the tree. Thankfully, there is a simple
solution. The splaying operation conveniently rotates the found node all the
way to the root. So, the SplayTreeNode.get method will return the new
root of the subtree, and the SplayTree.get returns the value at the root.

$$
bf = len(x.left)-len(x.right) \in \{-1, 0, 1\}\\
or\\
bf = |len(x.left)-len(x.right)| \leq 1\\
where \ bf \ is \ the \ \text{\textbf{balance factor}}
$$

In [5]:
from ds2.orderedmapping import BalancedBST, BalancedBSTNode

class SplayTreeNode(BalancedBSTNode):
    def newnode(self, key, value):
        return SplayTreeNode(key, value)
    
    def splayup(self, key):
        newroot = self
        if key < self.key:
            if key < self.left.key:
                newroot = self.rotateright().rotateright()
            elif key > self.left.key:
                self.left = self.left.rotateleft()
                newroot = self.rotateright()
        elif key > self.right:
            if key > self.right.key:
                newroot = self.rotateleft().rotateleft()
            elif key < self.right.key:
                self.right = self.right.rotateright()
                newroot = self.rotateleft()
        return newroot
    
    def put(self, key, value):
        newroot =  BalancedBSTNode.put(self, key, value)
        return newroot.splayup(key)
    
    def get(self, key):
        if key == self.key:
            return self
        elif key < self.key and self.left:
            self.left = self.left.get(key)
        elif key > self.key and self.right:
            self.right = self.right.get(key)
        else:
            raise KeyError
        return self.splayup(key)
    
class SplayTree(BalancedBST):
    Node = SplayTreeNode

    def splayup(self, key):
        if key < self._root.key:
            self._root = self._root.rotateright()
        if key > self._root.key:
            self._root = self._root.rotateleft()

    def get(self, key):
        if self._root is None: raise KeyError
        self._root = self._root.get(key)
        self.splayup(key)
        return self._root.value
    
    def put(self, key, value):
        BalancedBST.put(self, key, value)
        self.splayup(key)